# Finding the Optimal Split Between BSc and MSc Loans

This notebook analyzes the optimal way to repay two student loans — one for a BSc degree and one for an MSc degree — using a fixed total monthly payment. The objective is to minimize the total repayment time and/or the total interest paid by experimenting with different ways to split the payment between the two loans.

We simulate concurrent repayment, where any leftover payment from one loan (once paid off) is reallocated to the other. The notebook includes two main approaches:

1. **Fixed payments scenario** – where we simulate repayment of each loan separately.
2. **Dynamic optimization scenario** – where we try various payment splits to find the best strategy.


**Loan Parameters and Setup for Simulation**

We define:
- Total monthly payment capacity.
- Loan principals and interest rates for BSc and MSc loans.
- Monthly interest rates derived from annual interest.

To determine the best way to split the payment, we iterate over possible BSc allocations (`payment_bsc`) and derive the corresponding MSc amount. We simulate each configuration and retain the one with the lowest number of repayment months (with a fallback option to compare interest paid).

Important constraints:
- Monthly payment to each loan must be greater than its monthly interest — otherwise, the principal will never decrease.

In [ ]:
def simulate_loan(principal, monthly_interest_rate, monthly_payment):
    """
    Simulate the repayment of a loan.

    Parameters:
      principal (float): The initial loan amount.
      annual_interest_rate (float): Annual interest rate (in percent, e.g., 7 for 7%).
      monthly_payment (float): The amount paid each month.

    Returns:
      months (int): Number of months needed to pay off the loan.
      total_interest (float): Total interest paid over the life of the loan.
    """
    total_interest = 0.0
    months = 0

    # Check if the monthly payment is enough to cover at least the interest.
    if monthly_payment <= principal * monthly_interest_rate:
        print(principal * monthly_interest_rate)
        raise ValueError("Monthly payment is too low to cover even the monthly interest. "
                         "The loan will never be paid off.")

    while principal > 0:
        # Calculate interest for the current month
        interest = principal * monthly_interest_rate
        # Update principal: add interest, subtract payment
        principal = principal + interest - monthly_payment
        # Accumulate total interest paid
        total_interest += interest
        months += 1

        # If the last payment overpays, we can adjust (optional)
        if principal < 0:
            break

    return months, total_interest

## Scenario 1: Minimum Required Payments

This section simulates each loan independently using only the minimum required payments — just enough to gradually pay off each loan.

This allows us to:
- Establish a baseline of how long each loan would take to repay.
- Evaluate how much interest would accumulate under minimum effort.

We use the `simulate_loan` function here to isolate and study each loan’s behavior independently.

In [8]:
principal_bsc = 34767.08 
principal_msc = 12108.60  
monthly_rate_bsc = 4.3 / 100 / 12
monthly_rate_msc = 7.3 / 100 / 12
monthly_payment_bsc = 220
monthly_payment_msc = 120

months_bsc, total_interest_bsc = simulate_loan(principal=principal_bsc, monthly_interest_rate=monthly_rate_bsc, monthly_payment=monthly_payment_bsc)
months_msc, total_interest_msc = simulate_loan(principal=principal_msc, monthly_interest_rate=monthly_rate_msc, monthly_payment=monthly_payment_msc)

print("BSC")
print(f"Number of months: {months_bsc}")
print(f"Number of years: {months_bsc/12}")
print(f"Total amount of interst paid: {total_interest_bsc}")

print("MSC")
print(f"Number of months: {months_msc}")
print(f"Number of years: {months_msc/12}")
print(f"Total amount of interst paid: {total_interest_msc}")

BSC
Number of months: 234
Number of years: 19.5
Total amount of interst paid: 16612.11359556322
MSC
Number of months: 157
Number of years: 13.083333333333334
Total amount of interst paid: 6717.788970450137


## Scenario 2: Paying Extra Each Month

In this scenario, we simulate a strategy where we pay more than the minimum requirement: an additional 100, split between the two loans.

This helps us understand:
- The impact of additional payments on total interest.
- How much the loan duration is reduced compared to the baseline.

We again simulate both loans independently but with boosted payments.

In [ ]:
principal_bsc = 34767.08 
principal_msc = 12108.60  
monthly_rate_bsc = 4.3 / 100 / 12
monthly_rate_msc = 7.3 / 100 / 12
monthly_payment_bsc = 320
monthly_payment_msc = 220

months_bsc, total_interest_bsc = simulate_loan(principal=principal_bsc, monthly_interest_rate=monthly_rate_bsc, monthly_payment=monthly_payment_bsc)
months_msc, total_interest_msc = simulate_loan(principal=principal_msc, monthly_interest_rate=monthly_rate_msc, monthly_payment=monthly_payment_msc)

print("BSC")
print(f"Number of months: {months_bsc}")
print(f"Number of years: {months_bsc/12}")
print(f"Total amount of interst paid: {total_interest_bsc}")

print("MSC")
print(f"Number of months: {months_msc}")
print(f"Number of years: {months_msc/12}")
print(f"Total amount of interst paid: {total_interest_msc}")

BSC
Number of months: 1594
Number of years: 132.83333333333334
Total amount of interst paid: 164449.80987678742
MSC
Number of months: 43
Number of years: 3.5833333333333335
Total amount of interst paid: 1664.6232815408466


## Optimization: Finding the Best Split of Payments

In this block, we test various splits of the total payment between BSc and MSc loans. For each allocation:
- We simulate the joint repayment using `simulate_loans`.
- We ensure each allocation satisfies the minimum interest coverage.
- We track the combination that results in the fewest repayment months.

The result tells us the **optimal payment allocation** strategy that minimizes the total time needed to pay off both loans.

In [55]:
def simulate_loans(principal_bsc, principal_msc, 
                   monthly_rate_bsc, monthly_rate_msc, 
                   payment_bsc, payment_msc, total_payment):
    """
    Simulate paying two loans concurrently with a fixed total monthly payment.
    If one loan is paid off, its allocated payment is added to the other loan.
    
    Returns:
        months: Total months needed to pay off both loans.
        total_interest: Total interest paid over the life of the loans.
    """
    months = 0
    total_interest = 0.0

    # Loop until both loans are fully paid off.
    while principal_bsc > 1e-2 or principal_msc > 1e-2:
        months += 1
        
        # Accrue interest for each loan that is still active.
        if principal_bsc > 0:
            interest_bsc = principal_bsc * monthly_rate_bsc
            principal_bsc += interest_bsc
            total_interest += interest_bsc
        else:
            interest_bsc = 0
        
        if principal_msc > 0:
            interest_msc = principal_msc * monthly_rate_msc
            principal_msc += interest_msc
            total_interest += interest_msc
        else:
            interest_msc = 0
        
        # Reallocate payment: if one loan is paid off, the full total_payment goes to the other.
        if principal_bsc <= 0 and principal_msc > 0:
            pay_bsc = 0
            pay_msc = total_payment
        elif principal_msc <= 0 and principal_bsc > 0:
            pay_bsc = total_payment
            pay_msc = 0
        else:
            pay_bsc = payment_bsc
            pay_msc = payment_msc
        
        # Apply payments (ensure we don't go below zero).
        principal_bsc = max(0, principal_bsc - pay_bsc)
        principal_msc = max(0, principal_msc - pay_msc)
    
    return months, total_interest

In [ ]:
# Loan parameters
total_payment = 450
principal_bsc = 3767.08 
principal_msc = 12108.60  
monthly_rate_bsc = 4.3 / 100 / 12
monthly_rate_msc = 7.3 / 100 / 12

# To find the best initial split we test various values for the payment to BSc.
best = None

for payment_bsc in range(120, 140):  # Trying different splits for BSc loan.
    payment_msc = total_payment - payment_bsc

    # Check that each monthly payment is at least more than the monthly interest,
    # otherwise the principal will never decrease.
    if payment_bsc <= principal_bsc * monthly_rate_bsc or payment_msc <= principal_msc * monthly_rate_msc:
        continue

    # Run the simulation with this payment split.
    months, interest = simulate_loans(principal_bsc, principal_msc, 
                                      monthly_rate_bsc, monthly_rate_msc, 
                                      payment_bsc, payment_msc, total_payment)
    
    # Record the best result (e.g. minimal months).
    if best is None or months < best["months"]:
        best = {"months": months,
                "total_interest": interest,
                "payment_bsc": payment_bsc,
                "payment_msc": payment_msc}

if best:
    print(f"Optimal payment BSc: {best['payment_bsc']}")
    print(f"Optimal payment MSc: {best['payment_msc']}")
    print(f"Total months to pay off: {best['months']}")
    print(f"Total interest paid: {best['total_interest']:.2f}")
else:
    print("No valid payment split found.")

Optimal payment BSc: 120
Optimal payment MSc: 330
Total months to pay off: 40
Total interest paid: 1856.26
